### Импорт данных из xlsx в базу

In [1]:
import mysql.connector as mysql
import pandas as pd

con = mysql.connect(user = 'root', password = 'XxVJT7cd', 
                    host = 'localhost', database = 'Collection')
cur = con.cursor(dictionary=True)

In [2]:
#roles
df = pd.read_excel (r'/Users/margaritaberseneva/Desktop/db.xlsx', sheet_name=0)

In [3]:
for row in df.head():
     print(row)

RoleID
RoleName
RoleDesc
isSuperVisor
isBoss
IsActive


In [4]:
for i, j in df.iterrows(): 
    req = """INSERT INTO `Collection`.`roles` (`ID`, `Name`, `RoleDesc`, `IsBoss`, `IsActive`) VALUES """
    reqs = """ ( '{}','{}','{}','{}','{}')""".format(j.RoleID, j.RoleName, j.RoleDesc, j.isBoss, j.IsActive)
    req += reqs
    cur.execute(req)
    con.commit()

In [5]:
#statuses
df = pd.read_excel (r'/Users/margaritaberseneva/Desktop/db.xlsx', sheet_name=1)

In [6]:
for row in df.head():
     print(row)

ID
BPKindID
TypeID
Name
isEnabled
isEnding
IsHold
IsVerification


In [7]:
for i, j in df.iterrows(): 
    req = """INSERT INTO `Collection`.`statuses` (`ID`, `TypeID`, `Name`, `IsActive`, `IsEnding`, `IsHold`) VALUES """
    reqs = """ ( '{}','{}','{}','{}','{}','{}')""".format(j.ID, j.TypeID, j.Name, j.isEnabled, j.isEnding, j.IsHold)
    req += reqs
    cur.execute(req)
    con.commit()

In [22]:
#actions
df = pd.read_excel (r'/Users/margaritaberseneva/Desktop/db.xlsx', sheet_name=2)

In [23]:
for row in df.head():
     print(row)

ID
BPKindID
TypeID
Name
isEnabled
FromStatusID
ToStatusID
ProcessName


In [27]:
for i, j in df.iterrows(): 
    req = """INSERT INTO `Collection`.`actions` (`ID`, `TypeID`, `Name`, `IsActive`, `FromStatusID`, `ToStatusID`) VALUES """
    reqs = """( '{}', '{}','{}','{}','{}','{}')""".format(j.ID, j.TypeID, j.Name, j.isEnabled, j.FromStatusID, j.ToStatusID)
    req += reqs
    cur.execute(req)
    con.commit()

In [11]:
#confirmparams
df = pd.read_excel (r'/Users/margaritaberseneva/Desktop/db.xlsx', sheet_name=3)

In [12]:
for row in df.head():
     print(row)

ID
RoleID
StateID
IsKickback
IsReconfirm
titleText
IsActive


In [13]:
for i, j in df.iterrows(): 
    req = """INSERT INTO `Collection`.`confirmparams` (`ID`, `RoleID`, `StateID`, `Name`, `IsActive`) VALUES """
    reqs = """ ( '{}','{}','{}','{}','{}')""".format(j.ID, j.RoleID, j.StateID, j.titleText, j.IsActive)
    req += reqs
    cur.execute(req)
    con.commit()

In [18]:
#confirmbuttons
df = pd.read_excel (r'/Users/margaritaberseneva/Desktop/db.xlsx', sheet_name=4)

In [19]:
for row in df.head():
     print(row)

ID
ConfirmParamsID
ButtonTypeID
ConfirmSelectBPUserID
ActionID
ToStatusID
ButtonText
IsActive


In [20]:
for i, j in df.iterrows(): 
    req = """INSERT INTO `Collection`.`confirmbuttons` (`ID`, `ButtonTypeID`, `ConfirmParamsID`, `ActionID`, `ToStatusID`, `Name`, `IsActive`) VALUES """
    reqs = """ ( '{}','{}','{}', '{}','{}','{}','{}')""".format(j.ID, j.ButtonTypeID, j.ConfirmParamsID, j.ActionID, j.ActionID, j.ButtonText, j.IsActive)
    req += reqs
    cur.execute(req)
    con.commit()

In [21]:
cur.execute("SELECT COUNT(1) FROM confirmbuttons")
cur.fetchall()

[{'COUNT(1)': 8040}]

### Интерфейс

In [2]:
import ipywidgets as widgets

In [3]:
def viewing():
    view = widgets.Text(placeholder='введите название экшена или 1 для всех', description='', disabled=False)
    display(view)
    return view

In [4]:
def adding():
    add_what = widgets.Text(placeholder='введите точное название одной роли здесь', description='', disabled=False)
    display(add_what)
    add_to = widgets.Text(placeholder='введите точное название одного статуса здесь', description='', disabled=False)
    display(add_to)
    return add_what, add_to

In [5]:
def modifying():
    modify_from = widgets.Select(options=[('кнопку', 'confirmbuttons'), ('роль', 'roles')], value='confirmbuttons', description='Изменить ...', disabled=False)
    display(modify_from)
    modify_where = widgets.Text(placeholder='введите название здесь', description='', disabled=False)
    display(modify_where)
    return modify_from, modify_where

In [6]:
def deleting():
    delete_from = widgets.Select(options=[('кнопку', 'confirmbuttons'), ('роль', 'roles')], value='confirmbuttons', description='Удалить ...', disabled=False)
    display(delete_from)
    delete_where = widgets.Text(placeholder='введите название здесь', description='', disabled=False)
    display(delete_where)
    return delete_from, delete_where

#### Просмотр

In [7]:
# просмотр свзяки кнопка-экшн-статусы
# например для экшена Договор подписан
view = viewing()

Text(value='', placeholder='введите название экшена или 1 для всех')

In [8]:
if view.value == '1':
    select = '''SELECT actions.id action_id, actions.name action_name,
                       buttons.id button_id, buttons.name button_name,
                       from_s.id from_status_id, from_s.name from_status_name,
                       to_s.id to_status_id, to_s.name to_status_name
                FROM actions
                JOIN statuses as from_s on actions.FromStatusID = from_s.id 
                JOIN statuses as to_s on actions.ToStatusID = to_s.id
                JOIN confirmbuttons as buttons on actions.id = buttons.actionid
                WHERE from_s.isactive = 1 and actions.isactive = 1 and buttons.isactive = 1'''
else:
    select = '''SELECT actions.id action_id, actions.name action_name,
                       buttons.id button_id, buttons.name button_name,
                       from_s.id from_status_id, from_s.name from_status_name,
                       to_s.id to_status_id, to_s.name to_status_name
                FROM actions 
                JOIN statuses as from_s on actions.FromStatusID = from_s.id 
                JOIN statuses as to_S on actions.ToStatusID = to_S.id
                JOIN confirmbuttons as buttons on actions.id = buttons.actionid
                WHERE from_s.isactive = 1 and actions.isactive = 1 and buttons.isactive = 1 and actions.name like "%{}%"'''.format(view.value)
#print(select)
cur.execute(select)
cur.fetchall()

[{'action_id': 13737,
  'action_name': 'Договор подписан. УС Установлены',
  'button_id': 9907,
  'button_name': 'Установка УС завершена',
  'from_status_id': 5355,
  'from_status_name': 'Самоинкассация. Установка УС заввершена',
  'to_status_id': 4971,
  'to_status_name': 'Договор подписан клиентом'},
 {'action_id': 13995,
  'action_name': 'Договор подписан',
  'button_id': 13936,
  'button_name': 'Документы подписаны',
  'from_status_id': 5307,
  'from_status_name': 'Альфа-Касса. Подтверждение договора',
  'to_status_id': 5308,
  'to_status_name': 'Альфа-Касса. Документы подписаны'},
 {'action_id': 14984,
  'action_name': 'Договор подписан аутсорсом в регионе. Вернуть в УНДОИЛ',
  'button_id': 114831,
  'button_name': 'Договор подписан. Вернуть в УНДОИЛ',
  'from_status_id': 5710,
  'from_status_name': 'ОТОР. Обработка московского договора',
  'to_status_id': 5711,
  'to_status_name': 'Договор подписан в регионе'},
 {'action_id': 13344,
  'action_name': 'Договор подписан банком. Треб

#### Добавление

In [9]:
# добавление роли с привязкой к какому-нибудь статусу
# например роль Эксперт по согласованию договоров, статус Согласование договора
add_what, add_to = adding()

Text(value='', placeholder='введите точное название одной роли здесь')

Text(value='', placeholder='введите точное название одного статуса здесь')

In [10]:
select1 = "INSERT INTO roles (Name, ROleDesc, IsBoss, IsActive) VALUES ('{}', NULL, 0, 1)".format(add_what.value)
#print(select1)
cur.execute(select1)
select2 = "INSERT INTO confirmparams (RoleID, StateID, Name, IsActive) VALUES ((select id from roles where name like '{}'), (select id from statuses where name like '%{}%'), 'Действия роли на статусе', 1)".format(add_what.value, add_to.value)
#print(select2)
cur.execute(select2)
check = "select * from confirmparams where roleid in (select id from roles where name like '%{}%')".format(add_what.value)
#print(s)
cur.execute(check)
cur.fetchall()

[{'ID': 8648,
  'IsActive': 1,
  'Name': 'Действия роли на статусе',
  'RoleID': 1478,
  'StateID': 5661}]

#### Изменение

In [11]:
# изменение кнопки или роли, делаем кнопку невидимой или роль неактивной
# например для кнопки Вернуть на сверку, для роли Менеджер инкассации
modify_from, modify_where = modifying()

Select(description='Изменить ...', options=(('кнопку', 'confirmbuttons'), ('роль', 'roles')), value='confirmbuttons')

Text(value='', placeholder='введите название здесь')

In [12]:
if_select = "select isActive from {} where name like '%{}%'".format(modify_from.value, modify_where.value)
cur.execute(if_select)
a = cur.fetchall()
for item in a:
    for k, v in item.items():
        if v == 1:
            select = "UPDATE {} SET IsActive = 0 WHERE {}.id in (SELECT id FROM (SELECT * FROM {}) as A WHERE name like '%{}%')".format(modify_from.value, modify_from.value, modify_from.value, modify_where.value)
        elif v == 0:
            select = "UPDATE {} SET IsActive = 1 WHERE {}.id in (SELECT id FROM (SELECT * FROM {}) as A WHERE name like '%{}%')".format(modify_from.value, modify_from.value, modify_from.value, modify_where.value)
        cur.execute(select)

check = "SELECT * FROM {} where name like '%{}%'".format(modify_from.value, modify_where.value)
cur.execute(check)
cur.fetchall()

[{'ActionID': 15295,
  'ButtonTypeID': 1,
  'ConfirmParamsID': 7807,
  'ID': 115111,
  'IsActive': 0,
  'Name': 'Вернуть на сверку',
  'ToStatusID': 15295},
 {'ActionID': 15296,
  'ButtonTypeID': 1,
  'ConfirmParamsID': 6626,
  'ID': 115112,
  'IsActive': 0,
  'Name': 'Вернуть на сверку',
  'ToStatusID': 15296}]

#### Удаление

In [13]:
# удаление кнопки или роли из базы 
# например для кнопки Вернуть на сверку, для роли Менеджер инкассации
delete_from, delete_where = deleting()

Select(description='Удалить ...', options=(('кнопку', 'confirmbuttons'), ('роль', 'roles')), value='confirmbuttons')

Text(value='', placeholder='введите название здесь')

In [14]:
select = "DELETE FROM {} WHERE {}.id in (SELECT id FROM (SELECT * FROM {}) as A WHERE name like '%{}%')".format(delete_from.value, delete_from.value, delete_from.value, delete_where.value)
#print(select)
cur.execute(select)
check = "SELECT * FROM {} where name like '%{}%'".format(delete_from.value, delete_where.value)
cur.execute(check)
cur.fetchall()

[]